#  LangChain의 RAG 콤포넌트 - 문서 임베딩(Embeddings) 

### **학습 목표:**  임베딩 모델과 벡터 데이터베이스를 효과적으로 연동할 수 있다

### **실습 자료**: 

- data/transformer.pdf

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob  

from pprint import pprint  
import json

`(3) 문서 로드`

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/transformer.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

`(4) 텍스트 분할`

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,             # 청크 크기  
    chunk_overlap=200,           # 청크 중 중복되는 부분 크기
    length_function=len,         # 글자 수를 기준으로 분할
    separators=["\n\n", "\n", " ", ""],  # 구분자 - 재귀적으로 순차적으로 적용 
)

# PDF 문서를 텍스트로 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 문서 임베딩(Document Embedding)

- 개념: 
    - 텍스트를 벡터(숫자 배열)로 변환하는 과정
    - 문서의 의미적 특성을 수치화하여 컴퓨터가 이해하고 처리할 수 있는 형태로 변환 

- 목적:
    - 텍스트 간 유사도 계산 가능
    - 벡터 데이터베이스 저장 및 검색
    - 의미 기반 문서 검색 구현

- LangChain의 임베딩 모델 종류:
    - OpenAI 임베딩
    - HuggingFace 임베딩 
    - Ollama 임베딩

### 1. **OpenAI**

- LangChain에서 가장 널리 사용되는 임베딩 모델 중 하나

- 주요 특징:
    1. 고품질의 임베딩 생성
    2. 다양한 언어 지원 (다국어 지원)
    3. 일관된 성능
    4. 손쉬운 통합

- 사용시 주의사항:
    1. API 키 설정이 필요 (환경 변수 OPENAI_API_KEY)
    2. API 사용량에 따른 비용 발생
    3. 긴 텍스트는 자동으로 분할되지 않으므로 필요시 TextSplitter를 사용


- 모델별 특징

    | 모델명 | 가격 효율 (페이지/1달러) | 성능 (MTEB) | 최대 입력 | 기본 차원 | 특징 및 추천 |
    | :--- | :---: | :---: | :---: | :---: | :--- |
    | **`text-embedding-3-small`** | **62,500** | 62.3% | 8,191 | 1,536 | **[가성비]** ada-002 대비 5배 저렴하고 성능 우수. 일반적인 RAG 구축 시 1순위. |
    | **`text-embedding-3-large`** | 9,615 | **64.6%** | 8,191 | 3,072 | **[고성능]** 미세한 의미 차이 구분이 중요할 때 사용. 차원 축소 기능 지원. |
    | **`text-embedding-ada-002`** | 12,500 | 61.0% | 8,191 | 1,536 | **[레거시]** 기존에 널리 쓰이던 모델이나, 현재는 v3-small 사용을 권장함. |

> **💡 Tip:** `text-embedding-3` 계열은 **Matryoshka Embedding** 기술이 적용되어, 저장 공간 절약을 위해 임베딩 차원(예: 1536 → 512)을 줄여서 요청해도 성능 하락이 매우 적습니다.

`(1) embedding 모델`

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",  # 사용할 모델 이름
    dimensions=None, # 원하는 임베딩 차원 수를 지정 가능 (기본값: None)
    )

# 임베딩 객체 출력
embeddings_model

In [ ]:
# 임베딩 모델의 컨텍스트 길이 확인
embeddings_model.embedding_ctx_length

In [ ]:
# 임베딩 모델의 임베딩 차원 확인 - 기본값 (None)
embeddings_model.dimensions

In [ ]:
# OpenAIEmbeddings 모델 생성할 때 임베딩 차원을 지정하는 예시
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름
    dimensions=512, # 원하는 임베딩 차원 수를 지정 가능 (기본값: None)
    )

# 임베딩 모델의 임베딩 차원 확인 
embeddings_model.dimensions

In [ ]:
# OpenAIEmbeddings 모델 생성
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 객체 출력
embeddings_openai

`(2) embed_documents 사용`

In [ ]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_openai = embeddings_openai.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_openai)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_openai[0])}")
print(document_embeddings_openai[0])

`(3) embed_query 사용`

In [ ]:
embedded_query_openai = embeddings_openai.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query_openai)}")
print(embedded_query_openai)

`(4) 유사도 기반 검색`

In [ ]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

# 쿼리와 가장 유사한 문서 찾기 함수
def find_most_similar(
        query: str, 
        doc_embeddings: np.ndarray,
        embeddings_model  # 기본값 제거, 명시적 전달 강제
        ) -> tuple[str, float]:
    """
    쿼리와 가장 유사한 문서를 찾는 함수
    
    Args:
        query: 검색 쿼리 문자열
        doc_embeddings: 문서 임베딩 배열
        embeddings_model: 임베딩 모델 객체
    
    Returns:
        tuple: (가장 유사한 문서, 유사도 점수)
    """
    # 쿼리 임베딩: OpenAI 임베딩 사용 
    query_embedding = embeddings_model.embed_query(query)

    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # 가장 유사한 문서 인덱스 찾기
    most_similar_idx = np.argmax(similarities)

    # 가장 유사한 문서와 유사도 반환: 문서, 유사도
    return documents[most_similar_idx], similarities[most_similar_idx]

# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(
        query, 
        document_embeddings_openai, 
        embeddings_model=embeddings_openai
        )
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()
    

### 2. **Huggingface**

- LangChain에서 오픈소스 기반의 대표적인 임베딩 모델

- 주요 특징:
    1. 로컬 환경에서 실행 가능
    2. 다양한 사전학습 모델 지원
    3. 커스텀 모델 학습 및 적용 가능
    4. 무료 사용 가능 (API 비용 없음)

- 사용시 주의사항:
    1. 로컬 컴퓨팅 자원 필요 (CPU/GPU)
    2. 초기 모델 다운로드 시간 소요
    3. 메모리 사용량 고려 필요
    4. transformers 라이브러리 설치 필요

- 임베딩 벡터 특성:
    1. 모델별로 다양한 차원 제공 (128 ~ 1024)
    2. sentence-transformers 기반 구현
    3. BERT 계열 모델 구조 사용
    4. 코사인 유사도 기반 검색 최적화

- 대표적인 임베딩 모델:

    | 모델명 (Hugging Face ID) | 차원 | 언어 | 특징 및 추천 용도 |
    | :--- | :---: | :---: | :--- |
    | **`all-MiniLM-L6-v2`** | 384 | **영어** | **[영어 표준/경량]** 매우 빠르고 메모리 효율이 좋음. 영어 전용 검색/분류 작업의 입문용 모델. |
    | **`all-mpnet-base-v2`** | 768 | **영어** | **[영어 고성능]** MiniLM보다 느리지만, 문장의 뉘앙스를 가장 정확하게 포착함. 영어권 RAG의 표준. |
    | **`paraphrase-multilingual-MiniLM-L12-v2`** | 384 | 다국어 | **[다국어 경량]** 한국어를 포함한 50개국어 지원. 속도가 빨라 실시간 서비스에 적합. |
    | **`intfloat/multilingual-e5-large`** | 1024 | 다국어 | **[다국어 고성능]** 다국어 벤치마크 상위권 모델. (사용 시 `query:`, `passage:` 접두어 필요) |
    | **`BAAI/bge-m3`** | 1024 | 다국어 | **[한국어 최적]** 한국어 처리 성능이 매우 뛰어나며, 긴 문장(8192 토큰)도 처리 가능. |


`(1) embedding 모델`

- langchain_huggingface 설치 필요

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings  

# Hugging Face의 임베딩 모델 생성
embeddings_gemma = HuggingFaceEmbeddings(
    model_name="google/embeddinggemma-300m",          # 사용할 모델 이름 - 구글의 경량 임베딩 모델
    # model_kwargs={'device': 'cuda'}  # GPU 사용시
    # model_kwargs={'device': 'mps'}   # Mac M1 사용시
)

# 임베딩 객체 출력
embeddings_gemma

`(2) embed_documents 사용`

In [ ]:
# 문서 임베딩
document_embeddings_gemma = embeddings_gemma.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_gemma)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_gemma[0])}")
print(document_embeddings_gemma[0])

`(3) embed_query 사용`

In [ ]:
embedded_query = embeddings_gemma.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

`(4) 유사도 기반 검색`

In [ ]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings_gemma, embeddings_model=embeddings_gemma) 
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

### 3. **Ollama (로컬 실행 최적화)**

LangChain에서 로컬 LLM 및 임베딩 모델을 가장 손쉽게 실행할 수 있는 플랫폼입니다. 외부 API 전송 없이 로컬 자원(GPU/CPU)만 사용하므로 데이터 보안과 비용 절감에 최적화되어 있습니다.

**주요 특징:**
1. **완전한 로컬 실행:** 데이터가 외부로 유출되지 않아 기업 내부(On-premise) 구축에 적합.
2. **빠른 추론 속도:** C++ 기반의 런타임과 양자화(Quantization) 기술로 최적화됨.
3. **간편한 배포:** Docker 기반으로 모델 설치 및 실행이 매우 간단함 (`ollama pull 모델명`).

**사용 시 주의사항:**
1. **서버 실행 필수:** 백그라운드에서 `ollama serve`가 실행 중이어야 함.
2. **리소스 관리:** 고성능 모델(Large) 사용 시 충분한 RAM/VRAM 필요.
3. **API 설정:** LangChain 등에서 호출 시 엔드포인트(`localhost:11434`) 확인 필요.

**대표적인 임베딩 모델 비교:**

| 모델명 (Model Tag) | 차원 | 언어 | 특징 및 추천 용도 |
| :--- | :---: | :---: | :--- |
| **`nomic-embed-text`** | 768 | 영어 | **[Ollama 표준]** 긴 문맥(8192 토큰)을 지원하며, 오픈소스 중 밸런스가 가장 우수함. |
| **`mxbai-embed-large`** | 1024 | 영어 | **[SOTA 성능]** MTEB 리더보드 상위권 모델. 검색 정확도가 매우 높음. |
| **`snowflake-arctic-embed`** | 1024 | 영어 | **[검색 최적화]** Snowflake사가 RAG 및 대규모 검색 작업에 특화하여 설계함. |
| **`bge-m3`** | 1024 | **다국어** | **[한국어 추천]** Ollama에서 사용 가능한 가장 강력한 다국어/한국어 모델. |
| **`all-minilm`** | 384 | 영어 | **[초경량]** 속도가 매우 빠르고 CPU 환경에서도 부담 없이 실행 가능. |

**임베딩 벡터 특성:**
1. **고정 차원:** 모델별로 384~1024의 고정된 벡터 차원을 가짐.
2. **자동 양자화:** 원본 모델을 4비트(Q4_0) 등으로 압축하여 메모리 사용량을 대폭 줄임.

`(1) embedding 모델`

- langchain_ollama 설치 필요

In [ ]:
from langchain_ollama import OllamaEmbeddings 

# OllamaEmbeddings 모델 생성
# embeddings_ollama = OllamaEmbeddings(
#     model="nomic-embed-text",          # 사용할 모델 이름
#     base_url="http://localhost:11434"  # Ollama 서버 주소
# )
embeddings_ollama = OllamaEmbeddings(model="bge-m3")

# 임베딩 객체 출력
embeddings_ollama

`(2) embed_documents 사용`

In [ ]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_ollama = embeddings_ollama.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_ollama)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_ollama[0])}")
print(document_embeddings_ollama[0])

`(3) embed_query 사용`

In [ ]:
embedded_query = embeddings_ollama.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

`(4) 유사도 기반 검색`

In [ ]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings_ollama, embeddings_model=embeddings_ollama) 
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

# [실습 프로젝트]

1. OpenAI text-embedding-3-small 임베딩 모델을 초기화합니다. 
2. 임베딩 차원을 각각 512와 1536으로 구분하여 2개의 모델을 생성합니다. 
3. 아래 주어진 문장들의 임베딩을 생성합니다. (임베딩 차원이 512, 1536인 경우 각각 2개씩 생성)
   - 문장1: "인공지능은 현대 사회를 변화시키고 있다"
   - 문장2: "AI 기술이 우리의 미래를 바꾸고 있다"
4. 생성된 임베딩의 차원을 출력합니다. (임베딩 차원이 512, 1536인 경우를 각각 출력)
5. 두 문장 간의 코사인 유사도를 계산합니다. (임베딩 차원이 512, 1536인 경우를 각각 비교)
6. [추가] 임베딩 차원(512 vs 1536)에 따른 유사도 차이를 분석하고, 어떤 차원이 더 효과적인지 생각해보세요.

In [ ]:
# 여기에 코드를 작성하세요.